### What type of programmer has the highest level of salary.
In order to know what kind of programmer has the highest level of salary. We need to extract the field of salary and developer types. Developer types could be found using the following code:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from collections import defaultdict
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./survey_results_public.csv')
df.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
developer_type = df['DeveloperType'].value_counts().reset_index()

developer_type.head()

,index,DeveloperType
0,Web developer,4283
1,Web developer; Desktop applications developer,775
2,Web developer; Mobile developer,573
3,Desktop applications developer,561
4,Mobile developer,559
5,Other,463
6,Web developer; DevOps specialist,308
7,Web developer; Mobile developer; Desktop appli...,275
8,Embedded applications/devices developer,260
9,Web developer; Database administrator,189


In [11]:
def get_possible_types(df, typecol):
    '''
    INPUT:
    df - the pandas dataframe
    typecol - type column name
    
    OUTPUT:
    possible_types - a set of possible developer types
    '''
    possible_types = set()
    developer_type = df[typecol].value_counts().reset_index()
    for typestr in developer_type['index']:
        types = typestr.split(";")
        for devtype in types:
            possible_types.add(devtype)
        
    return possible_types

    
def total_count(df, col1, col2, look_for):
    '''
    INPUT:
    df - the pandas dataframe you want to search
    col1 - the column name you want to look through
    col2 - the column you want to count values from
    look_for - a list of strings you want to search for in each row of df[col]

    OUTPUT:
    new_df - a dataframe of each look_for with the count of how often it shows up
    '''
    new_df = defaultdict(int)
    #loop through list of ed types
    for val in look_for:
        #loop through rows
        for idx in range(df.shape[0]):
            #if the ed type is in the row add 1
            if val in df[col1][idx]:
                new_df[val] += int(df[col2][idx])
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = [col1, col2]
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df

def clean_and_plot(df, title='Developer type', plot=True):
    '''
    INPUT 
        df - a dataframe holding the CousinEducation column
        title - string the title of your plot
        plot - bool providing whether or not you want a plot back
        
    OUTPUT
        study_df - a dataframe with the count of how many individuals
        Displays a plot of pretty things related to the CousinEducation column.
    '''
    study = df['DeveloperType'].value_counts().reset_index()
    study.rename(columns={'index': 'type', 'DeveloperType': 'count'}, inplace=True)
    study_df = total_count(study, 'type', 'count', possible_vals)

    study_df.set_index('type', inplace=True)
    if plot:
        (study_df/study_df.sum()).plot(kind='bar', legend=None);
        plt.title(title);
        plt.show()
    props_study_df = study_df/study_df.sum()
    return props_study_df

    
possible_types = get_possible_types(df, 'DeveloperType'


{' Data scientist',
 ' Database administrator',
 ' Desktop applications developer',
 ' DevOps specialist',
 ' Developer with a statistics or mathematics background',
 ' Embedded applications/devices developer',
 ' Graphic designer',
 ' Graphics programming',
 ' Machine learning specialist',
 ' Mobile developer',
 ' Other',
 ' Quality assurance engineer',
 ' Systems administrator',
 'Data scientist',
 'Database administrator',
 'Desktop applications developer',
 'DevOps specialist',
 'Developer with a statistics or mathematics background',
 'Embedded applications/devices developer',
 'Graphic designer',
 'Graphics programming',
 'Machine learning specialist',
 'Mobile developer',
 'Other',
 'Quality assurance engineer',
 'Systems administrator',
 'Web developer'}